<a href="https://colab.research.google.com/github/Jaovitosr/Extrator-XML/blob/main/Leitor_xml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import xml.etree.ElementTree as ET
from xml.dom import minidom
import re
import csv
import pandas as pd

#Correção do formato do arquivo .xml gerado pelo iTrace#

In [ ]:
def alterar_nomes_xml(arquivo_xml):

    with open(arquivo_xml, 'r') as file:
        conteudo = file.read()

    padrao_abertura = r"<(\d+)>"
    padrao_fechamento = r"</(\d+)>"


    conteudo_modificado = re.sub(padrao_abertura, r"<prefixo_\1>", conteudo)

    conteudo_modificado = re.sub(padrao_fechamento, r"</prefixo_\1>", conteudo_modificado)

    conteudo_modificado = re.sub(r"<\?xml.*?\?>\s*", "", conteudo_modificado, count=1)

    conteudo_modificado = f'<?xml version="1.0" encoding="UTF-8"?>\n<root>\n{conteudo_modificado}\n</root>'

    with open('itraceFixed.xml', 'w') as file:
        file.write(conteudo_modificado)

alterar_nomes_xml('itrace_chrome.xml')

#Armazenamento dos dados em uma lista de listas#

In [ ]:
data = [
    ["area", "id", "timestamp"]
]

i = 1
j = 1
k = 1

with open("itraceFixed.xml", 'r', encoding='utf-8') as f:
    xml = minidom.parse(f)
    area = xml.getElementsByTagName("area")
    timestamp = xml.getElementsByTagName("current_timestamp")
    id = xml.getElementsByTagName("id")

    for tag in area:
        if i >= len(data):
          data.append([])
        data[i].insert(0, tag.firstChild.data)
        i += 1

    for tag in timestamp:
       if j >= len(data):
         data.append([])
       data[j].insert(2, tag.firstChild.data)
       j += 1

    for tag in id:
      if k >= len(data):
          data.append([])
      data[k].insert(1, tag.firstChild.data)
      k += 1

#Armazenamento dos dados em um .csv#

In [ ]:
csv_name = "data.csv"

with open (csv_name, mode='w', newline='') as data_csv:

  escritor_csv = csv.writer(data_csv)

  for linha in data:
      escritor_csv.writerow(linha)

#Uso do pandas para análise do .csv#

In [ ]:
df = pd.read_csv(csv_name)
df.head()

#Contagem do Fixation#


In [ ]:
dic = {}
timestamp = "timestamp"
next_area = None
time0 = 0
time1 = 0
time = 0

for index, row in df.iterrows():

    area = row['area']

    if index < len(df) - 1:
        proxima_linha = df.iloc[index + 1]
        next_area = proxima_linha['area']
        next_time = proxima_linha['timestamp']
    else:
      next_area = None

    if index > 0:
      linha_anterior = df.iloc[index - 1]
      previous_area = linha_anterior['area']

    if area in dic:

      if next_area != area and previous_area != area:
        time0 = int(row[timestamp])
        time1 = int(next_time)
        time = time1 - time0
        dic[area].append(time)

      elif next_area != area:
        time1 = int(row[timestamp])
        time = time1 - time0
        dic[area].append(time)

      elif next_area == area and previous_area != area:
        time0 = int(row[timestamp])
        time1 = int(row[timestamp])
        time = time1 - time0
        dic[area].append(time)

    else:
      dic[area] = []
      time0 = int(row[timestamp])

      if next_area != area and previous_area != area:
        time1 = int(next_time)
        time = time1 - time0
        dic[area].append(time)

for chave, lista_de_listas in dic.items():
    print(f'Chave: {chave}')
    for lista in lista_de_listas:
        print(lista)

#Armazenando a soma do Fixation num .csv




In [ ]:
with open('resultado.csv', mode='w', newline='') as arquivo_csv:
    writer = csv.writer(arquivo_csv)

    writer.writerow(['area', 'fixation'])

    for chave, valores in dic.items():

        soma = sum(valores)
        writer.writerow([chave, soma])